<h1><center> Generative Adversarial Networks </center></h1>

# Imports

The following code is written in Keras.

In [4]:
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.21.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


# Definition

In [5]:
img_rows = 28
img_cols = 28
channels = 1

In [6]:
img_shape = (img_rows, img_cols, channels)

# Build the generator and the discriminator

In [10]:
def build_generator():

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)
    
    return Model(noise, img)

In [11]:
def build_discriminator():

    img_shape = (img_rows, img_cols, channels)

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

# Set the model

Discriminator :

In [13]:
optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()
discriminator.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


Generator :

In [14]:
generator = build_generator()
generator.compile(
    loss='binary_crossentropy', 
    optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_8 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
__________

In [15]:
z = Input(shape=(100,))
img = generator(z)

In [17]:
# For the combined model we will only train the generator
discriminator.trainable = False

In [20]:
# The valid takes generated images as input and determines validity
valid = discriminator(img)

Combined model :

In [21]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

Create a function to save the images :

In [36]:
def save_imgs(epoch):
    
    # Predict from input noise
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    # Subplots
    fig, axs = plt.subplots(r, c)
    cnt = 0
    
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
            
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# Train the model

In [30]:
epochs = 30000
batch_size = 32
save_interval = 200

In [31]:
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

half_batch = int(batch_size / 2)

In [37]:
for epoch in range(epochs):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]

    noise = np.random.normal(0, 1, (half_batch, 100))

    # Generate a half batch of new images
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    noise = np.random.normal(0, 1, (batch_size, 100))

    # The generator wants the discriminator to label the generated samples as valid (ones)
    valid_y = np.array([1] * batch_size)

    # Train the generator
    g_loss = combined.train_on_batch(noise, valid_y)

    # Plot the progress
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % save_interval == 0:
        save_imgs(epoch)

/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.248077, acc.: 96.88%] [G loss: 1.080750]
1 [D loss: 0.212694, acc.: 96.88%] [G loss: 1.284979]
2 [D loss: 0.188605, acc.: 100.00%] [G loss: 1.388983]
3 [D loss: 0.168995, acc.: 96.88%] [G loss: 1.511297]
4 [D loss: 0.145460, acc.: 100.00%] [G loss: 1.637576]
5 [D loss: 0.135139, acc.: 100.00%] [G loss: 1.718239]
6 [D loss: 0.103809, acc.: 100.00%] [G loss: 1.861278]
7 [D loss: 0.096354, acc.: 100.00%] [G loss: 1.869770]
8 [D loss: 0.099087, acc.: 100.00%] [G loss: 1.946192]
9 [D loss: 0.071478, acc.: 100.00%] [G loss: 2.005804]
10 [D loss: 0.095299, acc.: 100.00%] [G loss: 2.106457]
11 [D loss: 0.089296, acc.: 100.00%] [G loss: 2.207582]
12 [D loss: 0.079833, acc.: 100.00%] [G loss: 2.272530]
13 [D loss: 0.066200, acc.: 100.00%] [G loss: 2.301576]
14 [D loss: 0.060797, acc.: 100.00%] [G loss: 2.483273]
15 [D loss: 0.054994, acc.: 100.00%] [G loss: 2.452242]
16 [D loss: 0.072225, acc.: 100.00%] [G loss: 2.471909]
17 [D loss: 0.062363, acc.: 100.00%] [G loss: 2.583466]
18 [D

148 [D loss: 0.312163, acc.: 81.25%] [G loss: 3.166536]
149 [D loss: 0.036019, acc.: 100.00%] [G loss: 4.173116]
150 [D loss: 0.088481, acc.: 96.88%] [G loss: 3.037829]
151 [D loss: 0.321910, acc.: 87.50%] [G loss: 3.381609]
152 [D loss: 0.173277, acc.: 90.62%] [G loss: 4.044949]
153 [D loss: 0.086098, acc.: 100.00%] [G loss: 3.527914]
154 [D loss: 0.141887, acc.: 90.62%] [G loss: 3.626070]
155 [D loss: 0.076216, acc.: 100.00%] [G loss: 3.383617]
156 [D loss: 0.110409, acc.: 96.88%] [G loss: 3.999754]
157 [D loss: 0.290373, acc.: 87.50%] [G loss: 3.578058]
158 [D loss: 0.263636, acc.: 87.50%] [G loss: 4.110421]
159 [D loss: 0.437620, acc.: 75.00%] [G loss: 2.529198]
160 [D loss: 0.121529, acc.: 96.88%] [G loss: 3.246523]
161 [D loss: 0.051655, acc.: 96.88%] [G loss: 4.410876]
162 [D loss: 0.122262, acc.: 93.75%] [G loss: 4.239532]
163 [D loss: 0.239613, acc.: 87.50%] [G loss: 3.150080]
164 [D loss: 0.094365, acc.: 100.00%] [G loss: 3.429491]
165 [D loss: 0.296596, acc.: 87.50%] [G loss

296 [D loss: 0.684326, acc.: 50.00%] [G loss: 0.755473]
297 [D loss: 0.685676, acc.: 46.88%] [G loss: 0.773323]
298 [D loss: 0.725884, acc.: 43.75%] [G loss: 0.758417]
299 [D loss: 0.705634, acc.: 46.88%] [G loss: 0.762198]
300 [D loss: 0.637575, acc.: 56.25%] [G loss: 0.815473]
301 [D loss: 0.627951, acc.: 62.50%] [G loss: 0.873930]
302 [D loss: 0.738828, acc.: 43.75%] [G loss: 0.767262]
303 [D loss: 0.680795, acc.: 50.00%] [G loss: 0.838570]
304 [D loss: 0.761805, acc.: 40.62%] [G loss: 0.740516]
305 [D loss: 0.816934, acc.: 31.25%] [G loss: 0.652110]
306 [D loss: 0.676924, acc.: 46.88%] [G loss: 0.729872]
307 [D loss: 0.692207, acc.: 46.88%] [G loss: 0.744445]
308 [D loss: 0.719997, acc.: 31.25%] [G loss: 0.720444]
309 [D loss: 0.736318, acc.: 43.75%] [G loss: 0.699308]
310 [D loss: 0.701618, acc.: 50.00%] [G loss: 0.686333]
311 [D loss: 0.674135, acc.: 46.88%] [G loss: 0.707235]
312 [D loss: 0.771126, acc.: 40.62%] [G loss: 0.653987]
313 [D loss: 0.701433, acc.: 43.75%] [G loss: 0.

444 [D loss: 0.645650, acc.: 59.38%] [G loss: 0.680709]
445 [D loss: 0.643754, acc.: 50.00%] [G loss: 0.687406]
446 [D loss: 0.648417, acc.: 53.12%] [G loss: 0.678998]
447 [D loss: 0.658027, acc.: 50.00%] [G loss: 0.683515]
448 [D loss: 0.690983, acc.: 46.88%] [G loss: 0.708079]
449 [D loss: 0.611889, acc.: 68.75%] [G loss: 0.694506]
450 [D loss: 0.643974, acc.: 50.00%] [G loss: 0.686462]
451 [D loss: 0.682325, acc.: 56.25%] [G loss: 0.677211]
452 [D loss: 0.668010, acc.: 56.25%] [G loss: 0.684630]
453 [D loss: 0.646855, acc.: 50.00%] [G loss: 0.698778]
454 [D loss: 0.648269, acc.: 56.25%] [G loss: 0.698525]
455 [D loss: 0.649327, acc.: 65.62%] [G loss: 0.688665]
456 [D loss: 0.662907, acc.: 59.38%] [G loss: 0.682756]
457 [D loss: 0.657573, acc.: 56.25%] [G loss: 0.702012]
458 [D loss: 0.670258, acc.: 46.88%] [G loss: 0.700016]
459 [D loss: 0.652795, acc.: 59.38%] [G loss: 0.713321]
460 [D loss: 0.654960, acc.: 53.12%] [G loss: 0.691680]
461 [D loss: 0.658164, acc.: 56.25%] [G loss: 0.

...

29304 [D loss: 0.716921, acc.: 46.88%] [G loss: 0.812917]
29305 [D loss: 0.627628, acc.: 71.88%] [G loss: 0.929210]
29306 [D loss: 0.632522, acc.: 56.25%] [G loss: 0.763567]
29307 [D loss: 0.749023, acc.: 43.75%] [G loss: 0.924508]
29308 [D loss: 0.639115, acc.: 65.62%] [G loss: 0.885269]
29309 [D loss: 0.596344, acc.: 71.88%] [G loss: 0.940900]
29310 [D loss: 0.660730, acc.: 56.25%] [G loss: 0.914730]
29311 [D loss: 0.743585, acc.: 43.75%] [G loss: 0.848588]
29312 [D loss: 0.737674, acc.: 53.12%] [G loss: 0.851867]
29313 [D loss: 0.671458, acc.: 59.38%] [G loss: 0.866580]
29314 [D loss: 0.694359, acc.: 53.12%] [G loss: 0.776076]
29315 [D loss: 0.686056, acc.: 53.12%] [G loss: 0.826901]
29316 [D loss: 0.631972, acc.: 65.62%] [G loss: 0.806123]
29317 [D loss: 0.647994, acc.: 59.38%] [G loss: 0.846280]
29318 [D loss: 0.677184, acc.: 53.12%] [G loss: 0.826755]
29319 [D loss: 0.732309, acc.: 56.25%] [G loss: 0.723624]
29320 [D loss: 0.632793, acc.: 71.88%] [G loss: 0.850040]
29321 [D loss:

29450 [D loss: 0.644548, acc.: 65.62%] [G loss: 0.856417]
29451 [D loss: 0.659204, acc.: 71.88%] [G loss: 0.858569]
29452 [D loss: 0.640044, acc.: 59.38%] [G loss: 0.800417]
29453 [D loss: 0.737000, acc.: 46.88%] [G loss: 0.850754]
29454 [D loss: 0.673444, acc.: 53.12%] [G loss: 0.857885]
29455 [D loss: 0.678459, acc.: 59.38%] [G loss: 0.818064]
29456 [D loss: 0.648874, acc.: 56.25%] [G loss: 0.845588]
29457 [D loss: 0.770686, acc.: 50.00%] [G loss: 0.808079]
29458 [D loss: 0.671811, acc.: 50.00%] [G loss: 0.848973]
29459 [D loss: 0.644518, acc.: 65.62%] [G loss: 0.841141]
29460 [D loss: 0.637400, acc.: 59.38%] [G loss: 0.782925]
29461 [D loss: 0.649473, acc.: 71.88%] [G loss: 0.881283]
29462 [D loss: 0.662382, acc.: 65.62%] [G loss: 0.837129]
29463 [D loss: 0.768606, acc.: 50.00%] [G loss: 0.788256]
29464 [D loss: 0.677972, acc.: 46.88%] [G loss: 0.763817]
29465 [D loss: 0.624806, acc.: 65.62%] [G loss: 0.782948]
29466 [D loss: 0.685452, acc.: 59.38%] [G loss: 0.820050]
29467 [D loss:

29594 [D loss: 0.730495, acc.: 37.50%] [G loss: 0.857046]
29595 [D loss: 0.735394, acc.: 37.50%] [G loss: 0.814811]
29596 [D loss: 0.651901, acc.: 56.25%] [G loss: 0.798920]
29597 [D loss: 0.666629, acc.: 59.38%] [G loss: 0.759581]
29598 [D loss: 0.628979, acc.: 68.75%] [G loss: 0.919022]
29599 [D loss: 0.608938, acc.: 71.88%] [G loss: 0.935382]
29600 [D loss: 0.708483, acc.: 46.88%] [G loss: 0.863285]
29601 [D loss: 0.664121, acc.: 53.12%] [G loss: 0.883405]
29602 [D loss: 0.574029, acc.: 81.25%] [G loss: 0.899605]
29603 [D loss: 0.685393, acc.: 53.12%] [G loss: 0.913327]
29604 [D loss: 0.711793, acc.: 50.00%] [G loss: 0.899121]
29605 [D loss: 0.744881, acc.: 37.50%] [G loss: 0.881099]
29606 [D loss: 0.705234, acc.: 62.50%] [G loss: 0.853030]
29607 [D loss: 0.676424, acc.: 59.38%] [G loss: 0.858996]
29608 [D loss: 0.641627, acc.: 65.62%] [G loss: 0.877626]
29609 [D loss: 0.650697, acc.: 62.50%] [G loss: 0.828855]
29610 [D loss: 0.678397, acc.: 62.50%] [G loss: 0.851608]
29611 [D loss:

29738 [D loss: 0.678631, acc.: 50.00%] [G loss: 0.867806]
29739 [D loss: 0.640296, acc.: 62.50%] [G loss: 0.859661]
29740 [D loss: 0.646138, acc.: 65.62%] [G loss: 0.813259]
29741 [D loss: 0.638232, acc.: 59.38%] [G loss: 0.791719]
29742 [D loss: 0.653215, acc.: 62.50%] [G loss: 0.800641]
29743 [D loss: 0.672761, acc.: 65.62%] [G loss: 0.798153]
29744 [D loss: 0.675209, acc.: 68.75%] [G loss: 0.755590]
29745 [D loss: 0.620159, acc.: 78.12%] [G loss: 0.756914]
29746 [D loss: 0.707342, acc.: 50.00%] [G loss: 0.828799]
29747 [D loss: 0.709913, acc.: 56.25%] [G loss: 0.776730]
29748 [D loss: 0.728150, acc.: 53.12%] [G loss: 0.801354]
29749 [D loss: 0.685641, acc.: 50.00%] [G loss: 0.840062]
29750 [D loss: 0.674113, acc.: 65.62%] [G loss: 0.854558]
29751 [D loss: 0.718373, acc.: 46.88%] [G loss: 0.764382]
29752 [D loss: 0.671116, acc.: 59.38%] [G loss: 0.863186]
29753 [D loss: 0.619521, acc.: 59.38%] [G loss: 0.929117]
29754 [D loss: 0.613894, acc.: 59.38%] [G loss: 0.848798]
29755 [D loss:

29883 [D loss: 0.732106, acc.: 37.50%] [G loss: 0.817529]
29884 [D loss: 0.614751, acc.: 68.75%] [G loss: 0.897279]
29885 [D loss: 0.671412, acc.: 62.50%] [G loss: 0.865258]
29886 [D loss: 0.652662, acc.: 56.25%] [G loss: 0.902305]
29887 [D loss: 0.611472, acc.: 65.62%] [G loss: 0.877991]
29888 [D loss: 0.747108, acc.: 56.25%] [G loss: 0.855691]
29889 [D loss: 0.693029, acc.: 62.50%] [G loss: 0.902789]
29890 [D loss: 0.661411, acc.: 59.38%] [G loss: 0.808114]
29891 [D loss: 0.683703, acc.: 56.25%] [G loss: 0.884637]
29892 [D loss: 0.668427, acc.: 62.50%] [G loss: 0.905635]
29893 [D loss: 0.761721, acc.: 46.88%] [G loss: 0.804349]
29894 [D loss: 0.705501, acc.: 62.50%] [G loss: 0.851405]
29895 [D loss: 0.761428, acc.: 46.88%] [G loss: 0.832434]
29896 [D loss: 0.673625, acc.: 65.62%] [G loss: 0.865263]
29897 [D loss: 0.591093, acc.: 68.75%] [G loss: 0.883000]
29898 [D loss: 0.633574, acc.: 59.38%] [G loss: 0.848658]
29899 [D loss: 0.742870, acc.: 50.00%] [G loss: 0.750527]
29900 [D loss:

# Wrapping it up in a class

In [7]:
class GAN():
    def __init__(self):
        # Initiate image size
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Initiate optimizer
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(
            loss='binary_crossentropy', 
            optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        plt.close()

In [ ]:
gan = GAN()
gan.train(epochs=30000, batch_size=32, save_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.868568, acc.: 28.12%] [G loss: 0.651218]


FileNotFoundError: [Errno 2] No such file or directory: 'gan/images/mnist_0.png'